In [ ]:
import cv2, os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras import *
from keras.models import *
from keras.layers import *
from keras.optimizers import *

In [ ]:
data = 'idd-lite_1/idd20k_lite/'

img_train = data + 'leftImg8bit/train/'
seg_train = data + 'gtFine/train/'

img_val = data + 'leftImg8bit/val/'
seg_val = data + 'gtFine/val/'

In [ ]:
mi, ma = 0, 6
n_classes = 7

In [ ]:
def getImageArr(path, width, height):
    img = cv2.imread(path, 1)
    img = np.float32(cv2.resize(img, (width, height))) / 127.5 - 1
    return img

def getSegmentationArr(path, nClasses, width, height): 
    seg_labels = np.zeros((height, width, nClasses))
    img = cv2.imread(path, 1)
    img = cv2.resize(img, (width, height))
    img = img[:, :, 0]
    for c in range(nClasses):
        seg_labels[:, :, c] = (img == c).astype(int)
    return seg_labels

In [ ]:
input_height, input_width = 224, 224
output_height, output_width = 224, 224

In [ ]:
X_train, y_train = [], []
X_val, y_val = [], []

In [ ]:
train_img = os.listdir(img_train)
train_img.sort()
train_seg = os.listdir(seg_train)
train_seg.sort()
val_img = os.listdir(img_val)
val_img.sort()
val_seg = os.listdir(seg_val)
val_seg.sort()

In [ ]:
print(train_img[: 5])
print(train_seg[: 5])
print(val_img[: 5])
print(val_seg[: 5])

In [ ]:
for im, seg in zip(train_img, train_seg):
    X_train.append(getImageArr(img_train + im, input_width, input_height))
    y_train.append(getSegmentationArr(seg_train + seg, n_classes, output_width, output_height)) 
  
for im, seg in zip(val_img, val_seg):
    X_val.append(getImageArr(img_val + im, input_width, input_height))
    y_val.append(getSegmentationArr(seg_val + seg, n_classes, output_width, output_height))

In [ ]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val),np.array(y_val)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    bn_axis = 3

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name = conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis = bn_axis, name = bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding = 'same', name = conv_name_base + '2b')(x)
    x = BatchNormalization(axis = bn_axis, name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name = conv_name_base + '2c')(x)
    x = BatchNormalization(axis = bn_axis, name = bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
  
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides = (2, 2)):
    filters1, filters2, filters3 = filters
    bn_axis = 3

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides = strides, name = conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis = bn_axis, name = bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding = 'same', name = conv_name_base + '2b')(x)
    x = BatchNormalization(axis = bn_axis, name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name = conv_name_base + '2c')(x)
    x = BatchNormalization(axis = bn_axis, name = bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides = strides,
                      name = conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis = bn_axis, name = bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)

    return x

In [ ]:
def ResNet50(input_tensor = None, input_shape = None, pooling = None, classes = 1000):
    img_input = Input(shape = input_shape)
    bn_axis = 3

    x = Conv2D(64, (7, 7), strides = (2, 2), padding = 'same', name = 'conv1')(img_input)
    x = BatchNormalization(axis = bn_axis, name = 'bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides = (2, 2), padding = "same")(x)

    x = conv_block(x, 3, [64, 64, 256], stage = 2, block = 'a', strides = (1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage = 2, block = 'b')
    x = identity_block(x, 3, [64, 64, 256], stage = 2, block = 'c')

    x = conv_block(x, 3, [128, 128, 512], stage = 3, block = 'a')
    x = identity_block(x, 3, [128, 128, 512], stage = 3, block = 'b')
    x = identity_block(x, 3, [128, 128, 512], stage = 3, block = 'c')
    x = identity_block(x, 3, [128, 128, 512], stage = 3, block = 'd')

    x = conv_block(x, 3, [256, 256, 1024], stage = 4, block = 'a')
    x = identity_block(x, 3, [256, 256, 1024], stage = 4, block = 'b')
    x = identity_block(x, 3, [256, 256, 1024], stage = 4, block = 'c')
    x = identity_block(x, 3, [256, 256, 1024], stage = 4, block = 'd')
    x = identity_block(x, 3, [256, 256, 1024], stage = 4, block = 'e')
    x = identity_block(x, 3, [256, 256, 1024], stage = 4, block = 'f')

    x = conv_block(x, 3, [512, 512, 2048], stage = 5, block = 'a')
    x = identity_block(x, 3, [512, 512, 2048], stage = 5, block = 'b')
    x = identity_block(x, 3, [512, 512, 2048], stage = 5, block = 'c')

    model = Model(img_input, x, name = 'resnet50')
    resnet50_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
    model.load_weights(resnet50_weights_path)

    return model

In [ ]:
def conv_block_bn(prevlayer, filters, prefix, strides = (1, 1)):
    conv = Conv2D(filters, (3, 3), padding = "same", kernel_initializer = "he_normal", strides = strides, name = prefix + "_conv")(prevlayer)
    conv = BatchNormalization(name = prefix + "_bn")(conv)
    conv = Activation('relu', name = prefix + "_activation")(conv)

    return conv

def conv_block_no_bn(prevlayer, filters, prefix, strides = (1, 1)):
    conv = Conv2D(filters, (3, 3), padding = "same", kernel_initializer = "he_normal", strides = strides, name = prefix + "_conv")(prevlayer)
    conv = Activation('relu', name = prefix + "_activation")(conv)

    return conv

In [ ]:
def unet_resnet(nClasses, input_shape):
    resnet_base = ResNet50(input_shape = input_shape)

    for l in resnet_base.layers:
        l.trainable = True

    conv1 = resnet_base.get_layer("activation_1").output
    conv2 = resnet_base.get_layer("activation_10").output
    conv3 = resnet_base.get_layer("activation_22").output
    conv4 = resnet_base.get_layer("activation_40").output
    conv5 = resnet_base.get_layer("activation_49").output

    up6 = concatenate([UpSampling2D()(conv5), conv4], axis = -1)
    conv6 = conv_block_bn(up6, 256, "conv6_1")
    conv6 = conv_block_bn(conv6, 256, "conv6_2")

    up7 = concatenate([UpSampling2D()(conv6), conv3], axis = -1)
    conv7 = conv_block_bn(up7, 64, "conv7_1")
    conv7 = conv_block_bn(conv7, 64, "conv7_2")

    up8 = concatenate([UpSampling2D()(conv7), conv2], axis = -1)
    conv8 = conv_block_bn(up8, 16, "conv8_1")
    conv8 = conv_block_bn(conv8, 16, "conv8_2")

    up9 = concatenate([UpSampling2D()(conv8), conv1], axis = -1)
    conv9 = conv_block_bn(up9, 8, "conv9_1")
    conv9 = conv_block_bn(conv9, 8, "conv9_2")

    up10 = UpSampling2D()(conv9)
    conv10 = conv_block_bn(up10, nClasses, "conv10_1")
    conv10 = conv_block_bn(conv10, nClasses, "conv10_2")
    conv10 = SpatialDropout2D(0.2)(conv10)

    x = Conv2D(nClasses, kernel_size = (1, 1), strides = (1, 1), activation = "softmax", name = "prediction")(conv10)
    model = Model(resnet_base.input, x)

    return model

In [ ]:
model = unet_resnet(nClasses = n_classes, input_shape = (input_width, input_height, 3))
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [ ]:
hist1 = model.fit(x = X_train, y = y_train, validation_data = (X_val, y_val), batch_size = 30, epochs = 30, verbose = 1)

In [ ]:
for key in ['loss', 'val_loss']:
    plt.plot(hist1.history[key], label = key)
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(X_val)
y_predi = np.argmax(y_pred, axis=3)
y_testi = np.argmax(y_val, axis=3)
print(y_testi.shape, y_predi.shape)

In [ ]:
def IoU(yi, y_predi):
    IoUs = []
    Nclass = 7
    for c in range(Nclass):
        TP = np.sum((yi == c) & (y_predi == c))
        FP = np.sum((yi != c) & (y_predi == c))
        FN = np.sum((yi == c) & (y_predi != c)) 
        IoU = TP / float(TP + FP + FN)
        print('class {:02.0f}: #TP={:6.0f}, #FP={:6.0f}, #FN={:5.0f}, IoU={:4.3f}'.format(c, TP, FP, FN, IoU))
        IoUs.append(IoU)
    mIoU = np.mean(IoUs)
    print('Mean IoU: {:4.3f}'.format(mIoU))

In [ ]:
IoU(y_testi, y_predi)

In [ ]:
model.save('u-net.h5')